## Email Reply Generator

This solution takes an email and generates an appropriate reply using OpenAI's GPT-4o-mini model.

**Parameters:**
- Email content (the message to reply to)
- User name (for the signature)

The code uses object-oriented design with separate methods for prompt creation, API calls, and output handling. The AI model intelligently determines the type of email (job posting, interview, etc.) and responds accordingly.

Future iterations will include improved error handling, type hints, and testing as I develop stronger Python skills.

In [ ]:
# Step 1: Create your prompts
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

class Solutions:
    """A class to generate email replies using OpenAI's GPT models."""
    
    def __init__(self):
        """Initialize the Solutions class with API credentials and prompts."""
        load_dotenv(override=True)
        self.api_key = os.getenv('OPENAI_API_KEY')
        
        if not self.api_key:
            print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
        elif not self.api_key.startswith("sk-proj-"):
            print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
        
        self.emailopenai = OpenAI()
        self.system_prompt = "You are a helpful assistant that can help in writing a reply to an email content. Make sure to keep the reply short and concise. Remove the AI fluff"
        self.user_prompt_template = """
            Original Email:
            ----
            {email_content}
            ----

            Write a polite reply email. Follow this exact structure:

            Subject: Re: <extract and use the original email subject>

            Hello <extract sender's first name>,

            <Write 1-2 sentences based on the email type:
            - For job postings: Politely thank them and accept, mentioning you're actively looking 
            - For job interviews: Acknowledge enthusiastically and ask about interview format, timing, and recommended preparation materials
            - For other emails: Respond appropriately and professionally
            - If unclear: Politely ask for more context>

            Thanks,
            {sender_email}
            """
    
    # Step 2: Make the messages list
    def messagesForEmail(self, emailContent, userName):
        """
        Create the message array for the OpenAI API call.
        
        Args:
            emailContent (str): The original email content
            userName (str): The name to use in the email signature
            
        Returns:
            list: Formatted messages for the API
        """
        user_prompt = self.user_prompt_template.replace("{email_content}", emailContent)
        user_prompt = user_prompt.replace("{sender_email}", userName)
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        return messages
    
    # Step 3: Call OpenAI
    def email_reply(self, emailContent, userName):
        """
        Generate an email reply using OpenAI's API.
        
        Args:
            emailContent (str): The original email content
            userName (str): The name to use in the email signature
            
        Returns:
            str: The generated email reply
        """
        response = self.emailopenai.chat.completions.create(
            model="gpt-4o-mini", 
            messages=self.messagesForEmail(emailContent, userName)
        )
        return response.choices[0].message.content
    
    def caller(self, emailContent, userName):
        """
        Main method to generate and print an email reply.
        
        Args:
            emailContent (str): The original email content
            userName (str): The name to use in the email signature
            
        Returns:
            str: The generated email reply
        """
        result = self.email_reply(emailContent, userName)
        print(result)
        return result


# Step 4: Test the solution
if __name__ == "__main__":
    """
    Test the email reply generator
    The caller() method takes two arguments:
    1. emailContent (str): The full text of the email you wish to reply
    2. signature (str) : The signature
    """
    # Define the email you want to reply to:
    email_content= """Thanks for using Cursor in 2025!
We've created a personal summary of how you used Cursor this year: which models you used most, how many tab completions and agent messages, and how you compare with other power users.
"""
    #  Define the sender's name , you can replace it with your signature example: name \n url
    signature="Shubham Rana \n https://shubhamrana.lovable.app/"
    # Initialises the solution class
    solutions = Solutions()
    # invoke the caller
    solutions.caller(emailContent=email_content, userName=signature)